In [1]:
# !pip install sqlalchemy

In [1]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain

In [2]:


# Load environment variables from .env file
load_dotenv()

# Fetch API key from environment variable
google_api = os.getenv("google")
# Initialize the LLM
llm = GoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=google_api)
# llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=palm)

db_path = "C:/Users/Omkar/OneDrive/Desktop/SQL_chat_assistant/demodb.db"
db_uri = f"sqlite:///{db_path}"

print(db_uri)

sqlite:///C:/Users/Omkar/OneDrive/Desktop/SQL_chat_assistant/demodb.db


In [3]:
# Create SQLDatabase instance with sample rows
db = SQLDatabase.from_uri(
    db_uri,
    include_tables=['Employees', 'Departments'],  # Include your table names
    sample_rows_in_table_info=3
)

# Print table info (will show schema + sample rows similar to your example)
print(db.table_info)


CREATE TABLE "Departments" (
	"ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Manager" TEXT NOT NULL, 
	PRIMARY KEY ("ID")
)

/*
3 rows from Departments table:
ID	Name	Manager
1	Sales	Alice
2	Engineering	Bob
3	Marketing	Charlie
*/


CREATE TABLE "Employees" (
	"ID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Department" TEXT NOT NULL, 
	"Salary" INTEGER NOT NULL, 
	"Hire_Date" TEXT NOT NULL, 
	PRIMARY KEY ("ID"), 
	FOREIGN KEY("Department") REFERENCES "Departments" ("Name")
)

/*
3 rows from Employees table:
ID	Name	Department	Salary	Hire_Date
1	Alice	Sales	50000	2021-01-15
2	Bob	Engineering	70000	2020-06-10
3	Charlie	Marketing	60000	2022-03-20
*/


In [6]:
def execute_query(question):
    try:
        # Generate SQL query
        query = chain.invoke({"question": question})
        print("\nRaw Generated Query:")
        print(query)

        # Clean the query
        cleaned_query = query.split("SQLQuery:")[1].strip()
        print("\nCleaned SQL Query:")
        print(cleaned_query)

        # Normalize the query for validation
        normalized_query = cleaned_query.strip().lower()

        # Check if the query is a SELECT statement or starts with a CTE (WITH)
        if not (
            normalized_query.startswith("select") or
            normalized_query.startswith("with")
        ):
            return "Only SELECT queries (including CTEs) are allowed. This query cannot be executed."


        # Execute the query
        result = db.run(cleaned_query)
        
        # Handle empty results
        if not result:
            return "No results found for your query."

        # Format the result using the LLM
        finalresult = llm("Frame a simple short sentence with the following answer: " + result + " with reference to the following question: " + question)
        return finalresult

    except Exception as e:
        # Handle invalid queries or execution errors
        error_message = str(e)
        if "no such table" in error_message.lower():
            return "Error: The specified table does not exist."
        elif "no such column" in error_message.lower():
            return "Error: The specified column does not exist."
        elif "syntax error" in error_message.lower():
            return "Error: The generated SQL query is invalid."
        else:
            return f"An error occurred: {error_message}"


In [7]:
# Example usage
question = "empoyee who hired before 1 jan 2020"
result = execute_query(question)
print("\nQuery Results:")
print(result)


Raw Generated Query:
Question: empoyee who hired before 1 jan 2020
SQLQuery: SELECT "Name" FROM "Employees" WHERE "Hire_Date" < '2020-01-01' LIMIT 5

Cleaned SQL Query:
SELECT "Name" FROM "Employees" WHERE "Hire_Date" < '2020-01-01' LIMIT 5


C:\Users\Omkar\AppData\Local\Temp\ipykernel_20840\239151751.py:32: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  finalresult = llm("Frame a simple short sentence with the following answer: " + result + " with reference to the following question: " + question)



Query Results:
David, Eve, Hank, Ivy, and Leo were hired before January 1, 2020.



In [9]:
#function only to generate Query
def Generate_query(question):
    try:
        # Generate SQL query
        query = chain.invoke({"question": question})
        print("\nRaw Generated Query:")
        print(query)

        # Clean the query
        cleaned_query = query.split("SQLQuery:")[1].strip()
        print("\nCleaned SQL Query:")
        print(cleaned_query)

        # Normalize the query for validation
        normalized_query = cleaned_query.strip().lower()

        # Check if the query is a SELECT statement or starts with a CTE (WITH)
        if not (
            normalized_query.startswith("select") or
            normalized_query.startswith("with")
        ):
            return "Only SELECT queries (including CTEs) are allowed. This query cannot be executed."


        # # Execute the query
        # result = db.run(cleaned_query)
        
        # # Handle empty results
        # if not result:
        #     return "No results found for your query."

        # # Format the result using the LLM
        # finalresult = llm("Frame a simple short sentence with the following answer: " + result + " with reference to the following question: " + question)
        return cleaned_query

    except Exception as e:
        # Handle invalid queries or execution errors
        error_message = str(e)
        if "no such table" in error_message.lower():
            return "Error: The specified table does not exist."
        elif "no such column" in error_message.lower():
            return "Error: The specified column does not exist."
        elif "syntax error" in error_message.lower():
            return "Error: The generated SQL query is invalid."
        else:
            return f"An error occurred: {error_message}"


# Example usage
question = "empoyee who hired before 1 jan 2020"
result = Generate_query(question)
print("\nQuery Results:")
print(result)



Raw Generated Query:
Question: empoyee who hired before 1 jan 2020
SQLQuery: SELECT "Name" FROM "Employees" WHERE "Hire_Date" < '2020-01-01' LIMIT 5

Cleaned SQL Query:
SELECT "Name" FROM "Employees" WHERE "Hire_Date" < '2020-01-01' LIMIT 5

Query Results:
SELECT "Name" FROM "Employees" WHERE "Hire_Date" < '2020-01-01' LIMIT 5


In [11]:
# Example: Check if the table "Employees" exists
tables = db.run("SELECT Name FROM Employees ORDER BY Salary DESC LIMIT 1")
print("Tables in the database:")
print(tables)

Tables in the database:
[('Rachel',)]


In [12]:
def main():
    print("Initializing Chatbot...")

    while True:
        print("\nSay 'exit' to quit the chatbot.")
        
        question = input("You: ")
        if question is None:
            continue
        if question.lower() == "exit":
            print("Chatbot: Take care of yourself, goodbye!")
            break
        result = execute_query(question)
        print(f"Chatbot: {result}")
        

if __name__ == "__main__":
    main()

Initializing Chatbot...

Say 'exit' to quit the chatbot.


You:  empolyee with max salary



Raw Generated Query:
Question: empolyee with max salary
SQLQuery: SELECT "Name" FROM "Employees" ORDER BY "Salary" DESC LIMIT 1

Cleaned SQL Query:
SELECT "Name" FROM "Employees" ORDER BY "Salary" DESC LIMIT 1
Chatbot: Rachel has the highest salary.


Say 'exit' to quit the chatbot.


You:  empolyee who works in IT department



Raw Generated Query:
Question: empolyee who works in IT department
SQLQuery: SELECT "Name" FROM "Employees" WHERE "Department" = 'IT' LIMIT 5

Cleaned SQL Query:
SELECT "Name" FROM "Employees" WHERE "Department" = 'IT' LIMIT 5


Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 11
}
].


Chatbot: Frank works in IT.


Say 'exit' to quit the chatbot.


You:  exit


Chatbot: Take care of yourself, goodbye!


### different langchain